In [1]:
#An implementation of Algorithm

In [2]:
import pandas as pd
import numpy as np
import heapq
import math
import time

In [3]:
# Read in the dataset
df = pd.DataFrame(pd.read_csv('../data/compas-binary.csv'))

In [4]:
x_all = df.as_matrix()[:,:13]

y = df.as_matrix()[:,13]

In [5]:
# Association Rule Mining (Only one feature)

#support
#supp = [(x[:,i]*y).mean() for i in range(13)]
#supp

In [6]:
#confidence
conf1 = [sum(x_all[:,i]*y)/sum(x_all[:,i]) for i in range(13)]
conf1

[0.3667168674698795,
 0.592274678111588,
 0.5300462249614792,
 0.5184782608695652,
 0.45773618016964024,
 0.32459016393442625,
 0.45069360675512665,
 0.44644229291532195,
 0.4233735747820255,
 0.4932330827067669,
 0.28986197049024276,
 0.37864823348694315,
 0.6614535418583257]

In [7]:
#confidence
conf0 = [sum((x_all[:,i]==0)*y)/sum((x_all[:,i]==0)) for i in range(13)]
conf0

[0.48557089084065247,
 0.4481314432989691,
 0.45573665707893896,
 0.45415065976281943,
 0.4676032110091743,
 0.4923509759099701,
 0.7527272727272727,
 0.7275,
 0.711558854718982,
 0.45544199390353235,
 0.5382854764877236,
 0.4822479928635147,
 0.37143460807099093]

In [8]:
#x_idx = [conf1[i]>0.5 or conf0[i]>0.5 for i in range(len(conf1))]
"""
Because Using both conf1 and conf0 would select out too many features, 
which is hard for the algorithm to run out,
just use conf1 to select out a small fraction of feature.
"""
#x_idx = [conf1[i]>0.5 for i in range(len(conf1))]
#x_idx[0] = True # in the CORELS paper, gender is an important feature, so I add it manually
#x_idx

'\nBecause Using both conf1 and conf0 would select out too many features, \nwhich is hard for the algorithm to run out,\njust use conf1 to select out a small fraction of feature.\n'

In [9]:
#select out these features
#x = x_all[:,x_idx]

In [10]:
#manaually select out 5 features, accoring to CORELS paper when lambda=0.01
# sex:Female, age:18-20,age:21-22, juvenile-crimes:=0, priors:>3
x_idx = [0,1,2,8,12]
x = x_all[:,x_idx]
x

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1],
       ...,
       [0, 0, 0, 1, 0],
       [1, 0, 0, 1, 0],
       [1, 0, 1, 1, 0]])

In [11]:
nrule = x.shape[1]
ndata = len(y)

In [12]:
"""
calculate z, which is for the equivalent points bound
z is the vector defined in algorithm 5 of the CORELS paper
z is a binary vector indicating the data with a minority lable in its equivalent set
"""
z = pd.DataFrame([-1]*ndata).as_matrix()
# enumerate through theses samples
for i in range(ndata):
    #if z[i,0]==-1, this sample i has not been put into its equivalent set
    if z[i,0] == -1:
        tag1 = np.array([True]*ndata)
        for j in range(nrule):
            rule_label = x[i][j]
            #tag1 indicates which samples have exactly the same features with sample i
            tag1 = (x[:,j] == rule_label)*tag1
            
        y_l = y[tag1]
        pred = int(y_l.sum()/len(y_l) > 0.5)
        #tag2 indicates the samples in a equiv set which have the minority label
        tag2 = (y_l != pred)
        z[tag1,0] = tag2

z

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [80]:
def calcul(antecedents, x, y, points_cap=tuple()):
    """
    Function for calculating the predictions, number of data captured,
    number of data incorrectly captured by the leaves, and b0 (defined in (28) in the CORELS paper).
    """
    prediction = []
    num_captured = []
    num_captured_incorrect = []
    B0 = [] # b0 is defined in (28)
    points_captured = []
    
    num = len(antecedents)
    
    for i in range(num):
        
        
        if (len(points_cap)>0):
            #when split a leaf, points_cap is the vector indicating points captured by that leaf
            tag = np.array(points_cap)
            rule_index = abs(antecedents[i][-1])-1
            rule_label = int(antecedents[i][-1]>0)
            tag = (x[:,rule_index] == rule_label)*tag
        else:
            #when initialize the queue
            rule_index = abs(antecedents[i][0])-1
            rule_label = int(antecedents[i][0]>0)
            tag = (x[:,rule_index] == rule_label)
        
        points_captured.append(tag)
        
        # the y's of these data captured by leaf antecedents[i]
        y_leaf = y[tag]
        
        #b0 is defined in (28)
        b0 = tag.dot(z)[0]/ndata
        B0.append(b0)
        
        
        
        num_cap = len(y_leaf)
        num_captured.append(num_cap)
        
        if len(y_leaf)>0:
            pred = int(y_leaf.sum()/len(y_leaf) > 0.5)
            prediction.append(pred)
            num_cap_incor = sum(y_leaf != pred)
            num_captured_incorrect.append(num_cap_incor)
        else:
            prediction.append(0)
            num_captured_incorrect.append(0)

    if num==1:
        return prediction[0], num_captured[0], num_captured_incorrect[0], B0[0], points_captured[0]
        
    return prediction, num_captured, num_captured_incorrect, B0, points_captured

In [34]:
[0]*2

[0, 0]

In [14]:
class CacheTree:
    """
    A tree data structure.
    prefix: a 2-d tuple to encode the leaves
    prediction: a list to record the predictions of leaves
    num_captured: a list to record number of data captured by the leaves
    num_captured_incorrect: a list to record number of data incorrectly captured by the leaves
    """
    def __init__(self, x, y, prefix,
                 lamb, prior_metric = None,
                 prediction=None,
                 num_captured=None,
                 num_captured_incorrect=None,
                 deadleaf = None,
                 splitleaf = None,
                 lbound=None,
                 B0 = None, points_cap = None):
        self.prefix = prefix
        self.prediction = prediction
        self.num_captured = num_captured
        self.num_captured_incorrect = num_captured_incorrect
        self.deadleaf = deadleaf #a list indicate which leaves will never be split (support bound)
        self.splitleaf = splitleaf #a queue of lists indicating which leaves will be split in next rounds (1 for split, 0 for not split)
        self.lbound = lbound #a list of lower bound
        self.B0 = B0 # a list of b0
        self.points_cap = points_cap #a 2-d list (nleaves by ndata) indicating which points are captured by each leaf
        
        ndata = len(y)
        l = len(prefix)
        if prediction==None:
            self.prediction, self.num_captured, self.num_captured_incorrect, self.B0, self.points_cap = calcul(self.prefix, x, y)
            self.deadleaf = [0]*l
            self.splitleaf = [[1]*l]
            self.lbound = [sum(self.num_captured_incorrect[:i]+self.num_captured_incorrect[i+1:])/ndata + lamb*l 
                           for i in range(l)]
        #print(self.prefix)
        #print(self.lbound)
        #print(self.splitleaf)
        # which metrics to use for the priority queue
        if (prior_metric=="curiosity"):
            self.metric = min([self.lbound[i]/((ndata-self.num_captured[i])/len(y)) 
                                  for i in range(l) if self.splitleaf[0][i]==1])
        elif (prior_metric=="bound"):
            self.metric = min([self.lbound[i]
                                  for i in range(l) if self.splitleaf[0][i]==1])
        elif (prior_metric=="entropy"): 
            self.p = [self.num_captured_incorrect[i]/self.num_captured[i] 
                      if self.num_captured[i]!=0 else 0 for i in range(l)]
            self.entropy = [(-self.p[i]*math.log2(self.p[i])-(1-self.p[i])*math.log2(1-self.p[i]))*self.num_captured[i] 
                            if self.p[i]!=0 and self.p[i]!=1 else 0 for i in range(l)]
            self.metric = min([sum(self.entropy[:i]+self.entropy[i+1:])/(ndata-self.num_captured[i]) for i in range(l)])
        elif (prior_metric=="gini"):
            self.p = [self.num_captured_incorrect[i]/self.num_captured[i] 
                      if self.num_captured[i]!=0 else 0 for i in range(l)]
            self.giniindex = [(2*self.p[i]*(1-self.p[i]))*self.num_captured[i] for i in range(l)]
            self.metric = min([sum(self.giniindex[:i]+self.giniindex[i+1:])/(ndata-self.num_captured[i]) for i in range(l)])
        else:
            self.metric = 0

            
    def get_prefix(self):
        # return the encoded tree
        return self.prefix
    
    def get_pred(self):
        # return a list of length len(prefix)
        # the predictions of all leaves
        return self.prediction
    
    def get_cap(self):
        # return a list of length len(prefix)
        # the number of captured points of all leaves
        return self.num_captured
    
    def get_ncc(self):
        # return a list of length len(prefix)
        # the number of incorrectly captured points of all leaves
        return self.num_captured_incorrect
    
    def get_deadleaf(self):
        # return a list of length len(prefix)
        # indicating whether or not the leaf is dead (because of the support bound)
        return self.deadleaf
    
    def get_splitleaf(self):
        # return a queue of lists of length len(prefix)
        # indicating whether or not the leaf will be split
        return self.splitleaf
    
    def set_deadleaf(self,i):
        # set leaf i to be dead
        self.deadleaf[i] = 1
        return
    
        
    def get_lbound(self):
        # return a list of length len(prefix)
        # the lower bound of the tree with leaf i as d0, the rest as dp
        return self.lbound
    
    
    def get_metric(self):
        # return the metric (to be used as metrics in priority queue)
        return self.metric
    
    def get_B0(self):
        # return a list of length len(prefix)
        # b0
        return self.B0
    
    def get_pc(self):
        # return a list of length len(prefix)
        # points captured
        return self.points_cap
    
    def __lt__(self, other):
        # define <, which will be used in the priority queue
        return self.metric<other.metric

In [82]:
# cache every leaf

class CacheLeaf:
    """
    
    """
    def __init__(self, prediction=None,
                 num_captured=None,
                 num_captured_incorrect=None,
                 B0 = None, 
                 points_cap = None):
        self.prediction = prediction
        self.num_captured = num_captured
        self.num_captured_incorrect = num_captured_incorrect
        self.B0 = B0 # a list of b0
        self.points_cap = points_cap #a 2-d list (nleaves by ndata) indicating which points are captured by each leaf

In [15]:
class Eliminate:
    """
    A data structure to record and identify
    whether a tree has been visited/pruned
    """
    def __init__(self, elim_dict = None):
        self.elim_dict = {} # record these trees we have visited
        
    def eliminate(self, prefix):
        self.elim_dict[tuple(sorted(prefix))] = 1
        
    def is_duplicated(self, prefix):
        # if a tree is in the self.elim_dict, then we have already visited it
        if tuple(sorted(prefix)) in self.elim_dict.keys():
            #print("Eliminated!")
            return True
        return False

In [16]:
def Risk(tree,ndata):
    return tree.get_lbound()[0]+(tree.get_ncc()[0])/ndata

In [17]:
def log(lines, lamb, tic, queue_size, prefix_old, tree_new, R, d_c, R_c):
    "log"
    t = tree_new.get_prefix()
    t_c = d_c.get_prefix()
    
    the_time = str(time.time()-tic)
    the_queue_size = str(queue_size)
    the_split_tree = str(prefix_old)
    the_new_tree = str(t)
    the_new_tree_length = str(len(t))
    the_new_tree_pred = str(tree_new.get_pred())
    the_new_tree_objective = str(R)
    the_best_tree = str(t_c)
    the_length = str(len(t_c))
    the_pred = str(d_c.get_pred())
    the_obj = str(R_c)
    the_lbound = str(d_c.get_lbound())
    the_accuracy = str(1-(R_c - lamb*len(t_c)))
    the_num_cap = str(d_c.get_cap())
    the_num_cap_inc = str(d_c.get_ncc())


    line = ";".join([the_time, the_queue_size, the_split_tree, 
                     the_new_tree, the_new_tree_length, the_new_tree_pred,the_new_tree_objective,
                     the_best_tree, the_length, the_pred, the_obj, 
                     the_lbound, the_accuracy, the_num_cap, the_num_cap_inc])
    lines.append(line)

In [51]:
tuple(np.sum(([1],[5],[6]), axis=1))

(1, 5, 6)

In [79]:
def bbound(x, y, lamb, prior_metric = None, MAXDEPTH = 4, niter=float('Inf')):
    """
    An implementation of Algorithm
    ## one copy of tree
    ## mark which leaves to be split
    """
    
    #Initialize best rule list and objective
    d_c = None
    R_c = 1

    nrule = x.shape[1]
    ndata = len(y)
    print("nrule:", nrule)
    print("ndata:", ndata)
    
    E = Eliminate()
    tic = time.time()
    
    lines = [] # a list for log
    leaves = {} # cache leaves

    # initialize the queue to include all trees of just one split
    queue = []
    for r in range(1, nrule+1):
        tree0 = CacheTree(prefix = ((-r,),(r,)), x = x, y = y, lamb=lamb, prior_metric=prior_metric)
        heapq.heappush(queue, (tree0.get_metric(),tree0))
        #queue.append(tree0)
        R = Risk(tree0,ndata)
        if R<R_c:
            d_c = tree0
            R_c = R

        #log(lines, lamb, tic, len(queue), tuple(), tree0, R, d_c, R_c) 
    
    COUNT = nrule #count the total number of trees in the queue
    while (queue) and COUNT<niter:
        #tree = queue.pop(0)
        (curio, tree)=heapq.heappop(queue)
        d = tree.get_prefix()
        
        
        #print("=======COUNT=======",COUNT)
        #print("d",d)
        #print("R",tree.get_lbound()[0]+(tree.get_ncc()[0])/len(y))
        
        # if we have visited this tree
        if E.is_duplicated(d):
            continue
        else:
            E.eliminate(d)
        
        # the leaves we are going to split
        split_next = tree.get_splitleaf().copy()
        spl = split_next.pop(0)
        
        # enumerate through all the leaves
        for i in range(len(d)):
            # if the leaf is dead, then continue
            if tree.get_deadleaf()[i]==1:
                continue
            
            #(Lower bound on antecedent support)
            # if this bound doesnot hold, set the leaf to be dead, and continue
            if tree.get_cap()[i]/ndata/2 < lamb:
                tree.set_deadleaf(i)
                #print("==============dead==============",i)
                continue
            
            # 0 for not split; 1 for split
            #if spl[0][i]==0:
            if spl[i]==0:
                continue

            d0 = d[i] #d0 is the leaf we are going to split
            dp = d[:i]+d[i+1:] #dp is the rest
            
            
            # Restrict the depth of the tree
            if len(d0)>=MAXDEPTH:
                continue
            
            # we are going to split leaf i, and get 2 new leaves
            # we will add the two new leaves to the end of the list
            splitleaf_list = [split_next[k][:i]+split_next[k][i+1:]+split_next[k][i:i+1]*2
                              for k in range(len(split_next))]
            
            
            
            lb = tree.get_lbound()[i] # the lower bound 
            b0 = tree.get_B0()[i] # the b0 defined in (28) of the paper
            
            
            
            # split the leaf d0 with feature j
            for j in range(1, nrule+1):
                if (j not in d0)and(-j not in d0):
                    # split leaf d0 with feature j, and get 2 leaves l1 and l2
                    l1 = d0+(-j,)
                    l2 = d0+(j,)
                    t = dp+(l1, l2) # t is the new tree
                    #print("t",t)

                    # if tree t is duplicated, continue
                    if E.is_duplicated(t):
                        continue
                    
                    pred_l = [0]*2
                    cap_l = [0]*2
                    incorr_l = [0]*2
                    B0_l = [0]*2
                    points_l = [[0]*ndata]*2
                    
                    # for the two new leaves, calculate their predictions, 
                    # num of data captured, num of data incorrectly captured, and b0
                    l1_sorted = tuple(sorted(l1))
                    l2_sorted = tuple(sorted(l2))
                    Cache_l1 = leaves.get(l1_sorted, None)
                    Cache_l2 = leaves.get(l2_sorted, None)
                    
                    i_points = tree.get_pc()[i]
                    
                    
                    if Cache_l1 != None:
                        pred_l[0], cap_l[0], incorr_l[0], B0_l[0], points_l[0] = Cache_l1.prediction, Cache_l1.num_captured, Cache_l1.num_captured_incorrect, Cache_l1.B0, Cache_l1.points_cap
                    else:
                        pred_l[0], cap_l[0], incorr_l[0], B0_l[0], points_l[0] = calcul((l1,),x,y,i_points)
                        leaves[l1_sorted] = CacheLeaf(pred_l[0], cap_l[0], incorr_l[0], B0_l[0], points_l[0])
                    
                    if Cache_l2 != None:
                        pred_l[1], cap_l[1], incorr_l[1], B0_l[1], points_l[1] = Cache_l2.prediction, Cache_l2.num_captured, Cache_l2.num_captured_incorrect, Cache_l2.B0, Cache_l2.points_cap
                    else:
                        pred_l[1], cap_l[1], incorr_l[1], B0_l[1], points_l[1] = calcul((l2,),x,y,i_points)
                        leaves[l2_sorted] = CacheLeaf(pred_l[1], cap_l[1], incorr_l[1], B0_l[1], points_l[1])
                    
                    
                    
                    # calculate the bounds for each leaves in the new tree
                    loss_l1 = (incorr_l[0])/ndata
                    loss_l2 = (incorr_l[1])/ndata
                    loss_d0 = tree.get_ncc()[i]/ndata
                    delta = loss_l1+loss_l2-loss_d0+lamb
                    old_lbound = tree.get_lbound()[:i]+tree.get_lbound()[i+1:]
                    new_lbound = [b+delta for b in old_lbound]+[tree.get_lbound()[i]+loss_l2+lamb,tree.get_lbound()[i]+loss_l1+lamb]
                    
                    splitleaf_array = np.array(splitleaf_list)
                    sl = splitleaf_list.copy()
                        
                    #(Lower bound on accurate antecedent support)
                    a_l = (sum(cap_l)-sum(incorr_l))/ndata - sum(cap_l)/ndata/2
                    
                    #binary vector indicating split or not
                    splitleaf1 = [1]*(len(t)) #all leaves labeled as to be split
                    splitleaf2 = [0]*(len(t)-2)+[1,1] #l1,l2 labeled as to be split
                    splitleaf3 = [1]*(len(t)-2)+[0,0] #dp labeled as to be split
                    
                    if lb+b0+lamb>=R_c or lb>=R_c:
                        # if equivalent points bound combined with the lookahead bound doesn't hold
                        # or if the hierarchical objective lower bound doesn't hold
                        # we need to split at least one leaf in dp
                        
                        if a_l < lamb:
                        # if the bound doesn't hold, we need to split the leaf l1/l2 further
                            
                            if len(splitleaf_list)>0:
                                split_l1_l2 = splitleaf_array[:,-1].sum()+splitleaf_array[:,-2].sum()
                                
                                # if dp will have been split
                                if splitleaf_array.sum()-split_l1_l2>0:
                                    
                                    # if l1/l2 will have been split
                                    if split_l1_l2>0:
                                        sl.append(splitleaf1)
                                    
                                    # if l1/l2 will not have been split, we need to split l1/l2
                                    else:
                                        sl.append(splitleaf2)
                                
                                # and we need to split leaves in dp, if dp will not have been split
                                else:
                                    
                                    # if l1/l2 will have been split
                                    if split_l1_l2>0:
                                        sl.append(splitleaf3)
                                    
                                    # if l1/l2 will not have been split, we need to split l1/l2
                                    else:
                                        sl.append(splitleaf2)
                                        sl.append(splitleaf3)
                            else:
                                sl.append(splitleaf2)
                                sl.append(splitleaf3)
                                
                            
                        else:
                            
                            if len(splitleaf_list)>0:
                                split_l1_l2 = splitleaf_array[:,-1].sum()+splitleaf_array[:,-2].sum()
                                
                                # if dp will have been split
                                if splitleaf_array.sum()-split_l1_l2>0:
                                    sl.append(splitleaf1)
                                
                                # and we need to split leaves in dp, if dp will not have been split
                                else:
                                    sl.append(splitleaf3)
                            else:
                                sl.append(splitleaf3)
                    else:
                        
                        if a_l < lamb:
                            # if the bound doesn't hold, we need to split the leaf l1/l2 further
                            
                            
                            if len(splitleaf_list)>0:
                                split_l1_l2 = splitleaf_array[:,-1].sum()+splitleaf_array[:,-2].sum()
                                
                                # if l1/l2 will have been split
                                if split_l1_l2>0:
                                    sl.append(splitleaf1)
                                    
                                # if l1/l2 will not have been split, we need to split l1/l2
                                else:
                                    sl.append(splitleaf2)
                            else:
                                sl.append(splitleaf2)
                                
                        else:
                            sl.append(splitleaf1)
                    
                    #construct the new tree
                    tree_new = CacheTree(x = x, y = y, prefix = t,
                                         prediction = tree.get_pred()[:i]+tree.get_pred()[i+1:]+pred_l,
                                         num_captured = tree.get_cap()[:i]+tree.get_cap()[i+1:]+cap_l,
                                         num_captured_incorrect = tree.get_ncc()[:i]+tree.get_ncc()[i+1:]+incorr_l,
                                         deadleaf = tree.get_deadleaf()[:i]+tree.get_deadleaf()[i+1:]+[0,0],
                                         splitleaf = sl,
                                         lbound = new_lbound,
                                         B0 = tree.get_B0()[:i]+tree.get_B0()[i+1:]+B0_l,
                                         lamb = lamb,
                                         prior_metric=prior_metric,
                                         points_cap = tree.get_pc()[:i]+tree.get_pc()[i+1:]+points_l
                                        )

                    #queue.append(tree_new)
                    heapq.heappush(queue, (tree_new.get_metric(),tree_new))
                    R = Risk(tree_new,ndata)
                    if R<R_c:
                        d_c = tree_new
                        R_c = R
                    
                    COUNT = COUNT+1
                    
                    #log(lines, lamb, tic, len(queue), d, tree_new, R, d_c, R_c)
                    
                   
                
                
    header = ['time', 'queue_size', 'split_tree', 'new_tree', 'new_tree_length', 'new_tree_prediction', 'new_tree_objective',
              'best_tree', 'best_tree_length', 'prediction', 'objective', 'lower_bound', 'accuracy', 
              'num_captured', 'num_captured_incorrect']
    
    fname = "_".join([str(nrule),str(ndata),prior_metric,str(lamb),".txt"])
    f = open(fname, 'w')
    f.write('%s\n' % ";".join(header))
    f.write('\n'.join(lines))
    f.close()


    print("d_c", d_c.get_prefix())
    print("R_c", R_c)
    print("COUNT", COUNT)
    return

#### compas-binary

In [19]:
import cProfile

In [85]:
cProfile.run("bbound(x[:,:], y, lamb=0.01, prior_metric=\"curiosity\")")

nrule: 5
ndata: 6907
d_c ((5,), (-5, -4), (-5, 4))
R_c 0.3748675256985667
COUNT 1987326
         176918722 function calls in 1135.408 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1987326    3.962    0.000    3.962    0.000 <ipython-input-14-87578bc8ffa3>:102(get_metric)
  4981775    9.346    0.000    9.346    0.000 <ipython-input-14-87578bc8ffa3>:106(get_B0)
  5961963   11.115    0.000   11.115    0.000 <ipython-input-14-87578bc8ffa3>:111(get_pc)
  9379008   22.981    0.000   22.981    0.000 <ipython-input-14-87578bc8ffa3>:116(__lt__)
        5    0.000    0.000    0.000    0.000 <ipython-input-14-87578bc8ffa3>:34(<listcomp>)
  1987326   66.692    0.000   95.309    0.000 <ipython-input-14-87578bc8ffa3>:41(<listcomp>)
  1987327    4.036    0.000    4.036    0.000 <ipython-input-14-87578bc8ffa3>:61(get_prefix)
  3974642    7.560    0.000    7.560    0.000 <ipython-input-14-87578bc8ffa3>:65(get_pred)
 12144580   22.003   

In [84]:
cProfile.run("bbound(x[:,:], y, lamb=0.01, prior_metric=\"gini\")")

nrule: 5
ndata: 6907
d_c ((-4,), (4, -5), (4, 5))
R_c 0.3748675256985667
COUNT 961709
         105676267 function calls in 661.616 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   961709    1.967    0.000    1.967    0.000 <ipython-input-14-87578bc8ffa3>:102(get_metric)
  2972513    5.527    0.000    5.527    0.000 <ipython-input-14-87578bc8ffa3>:106(get_B0)
  2885112    5.346    0.000    5.346    0.000 <ipython-input-14-87578bc8ffa3>:111(get_pc)
   163704    0.417    0.000    0.417    0.000 <ipython-input-14-87578bc8ffa3>:116(__lt__)
        5    0.000    0.000    0.000    0.000 <ipython-input-14-87578bc8ffa3>:34(<listcomp>)
   961709    8.694    0.000    8.694    0.000 <ipython-input-14-87578bc8ffa3>:53(<listcomp>)
   961709   13.851    0.000   13.851    0.000 <ipython-input-14-87578bc8ffa3>:55(<listcomp>)
   961709   50.257    0.000   87.845    0.000 <ipython-input-14-87578bc8ffa3>:56(<listcomp>)
   961710    1.911    

In [83]:
%%time
#all data

bbound(x[:,2:], y, lamb=0.01, prior_metric="curiosity")

nrule: 3
ndata: 6907
d_c ((3,), (-3, -2), (-3, 2))
R_c 0.3748675256985667
COUNT 104
CPU times: user 148 ms, sys: 4 ms, total: 152 ms
Wall time: 150 ms


In [ ]:
%%time
#all data, 5 features

bbound(x, y, lamb=0.01, prior_metric="bound")

In [ ]:
%%time
#all data, 5 features

bbound(x, y, lamb=0.01, prior_metric="entropy")

In [ ]:
%%time
#all data, 5 features

bbound(x, y, lamb=0.01, prior_metric="gini")